### 安装 git lfs

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs && git lfs install

### 下载模型

In [ ]:
!git clone https://huggingface.co/THUDM/chatglm-6b ../chatglm-6b

### 安装依赖

In [ ]:
!pip install -r ./requirement.txt

### 数据集准备 

准备 `.jsonl` 格式的数据放到 ./data 目录下。数据格式为：


{"q": "问题", "a": "回答"}


### 对数据集进行分词

为了避免每次训练的时都要重新对数据集分词，先分好词形成特征后保存成可直接用于训练的数据集。相关参数说明：

* model_checkpoint: 模型目录
* input_file:  ./data 目录下的数据集文件名
* prompt_key:  数据集中 prompt 对应的字段（这里是 q）
* target_key:  数据集中 completion 对应的字段（这里是 a)
* save_name:  数据集保存目录，分词后的数据保存在 ./data/tokenized_data 下
* max_seq_length:  文本最大长度

In [2]:
!CUDA_VISIBLE_DEVICES=0 python ./script/tokenize_dataset_rows.py \
    --input_file dataset.jsonl \
    --prompt_key q \
    --target_key a \
    --save_name dataset \
    --max_seq_length 2000 \
    --skip_overlength False

Generating train split: 0 examples [00:00, ? examples/s]Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

Generating train split: 1 examples [00:01,  1.22s/ examples]155 [00:00<?, ?it/s]
Generating train split: 229 examples [00:01, 245.78 examples/s]:00, 1292.95it/s]
Generating train split: 357 examples [00:01, 375.74 examples/s]:00, 1031.38it/s]
 32%|████████████▍                          | 367/1155 [00:00<00:00, 900.90it/s]
Generating train split: 491 examples [00:01, 496.28 examples/s]0:00, 905.24it/s]
Generating train split: 618 examples [00:01, 637.45 examples/s]:00, 1013.68it/s]
Generating train split: 755 examples [00:01, 786.85 examples/s]:00, 1124.49it/s]
Generating train split: 885 examples [00:02, 808.30 examples

### 使用 LoRA 微调

参数说明：

* tokenized_dataset: 分词后的数据集保存目录（即上一步 save_name 的值）
* tlora_rank: 设置 LoRA 的秩，推荐为4或8，显存够的话使用8
* tper_device_train_batch_size: 每块 GPU 上的 batch size
* tgradient_accumulation_steps: 梯度累加，可以在不提升显存占用的情况下增大 batch size
* tmax_steps: 训练步数
* tsave_steps: 多少步保存一次
* tsave_total_limit: 保存多少个checkpoint
* tlogging_steps: 多少步打印一次训练情况(loss, lr, etc.)
* toutput_dir: 模型文件保存地址

In [ ]:
# 删除上次的微调模型
# !rm -rf /mnt/workspace/glm-fine-tuning/weights

!CUDA_VISIBLE_DEVICES=0 python ./script/chatglm_lora_tuning.py \
    --tokenized_dataset dataset \
    --lora_rank 8 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --max_steps 5000 \
    --save_steps 200 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir ./weights/api-fn

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
'\nlen(dataset)=1155\n'
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████████████| 8/8 [00:08<00:00,  1.01s/it]
{'': 0}
You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
/home/pai/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.opt

### 加载微调模型

微调模型保存在上一步配置的 output_dir 目录下。至少需要其中的 adapter_model.bin、adapter_config.json 两个文件才能部署成功

### 启动 web 服务

In [ ]:
!python ./server/web.py

### 通过 API 服务测试模型

In [5]:
# 安装 pyngrok 用来暴露服务

!npm install -g localtunnel

/etc/dsw/node/bin/lt -> /etc/dsw/node/lib/node_modules/localtunnel/bin/lt.jsming action:finalize
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 10.96s


In [ ]:
!lt --port 6006

your url is: https://lucky-suns-allow.loca.lt


The dtype of attention mask (torch.int64) is not bool


[2023-07-13 17:53:42] ", prompt:"API：
{
  "name": "get_okr",
  "description": "查询用户okr",
  "parameters": {
    "username": {
      "type": "string",
      "description": "用户名"
    }
  }
}

上下文：
辰秋的okr", response:"'{\n"username": "辰秋"\n}'"
INFO:     140.205.118.27:0 - "POST / HTTP/1.1" 200 OK
[2023-07-13 17:54:10] ", prompt:"API：
{
  "name": "get_okr",
  "description": "查询用户okr",
  "parameters": {
    "userList": {
      "type": "List<string>",
      "description": "用户列表"
    }
  }
}

上下文：
我和默辰的okr", response:"'{\n"userList": ["我和默辰"]\n}'"
INFO:     140.205.118.94:0 - "POST / HTTP/1.1" 200 OK


In [3]:
# 后台运行 chatlm
get_ipython().system_raw("python ./server/api.py &")

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:35<00:00,  4.42s/it]
INFO:     Started server process [16331]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:6006 (Press CTRL+C to quit)
